In [1]:
#Let us first import necessary libararies
import numpy as np
import glob
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Data Conversion

The lidar data from the KITTI dataset is in `.bin` format which is currently not an accepted format in Open3D, hence this is converted to its `.pcd` equivalent. This has already been done for you and is located in the DATA folder of the main repo. However, should you come to work with the KITTI dataset yourself, here is the code to convert `.bin` files to `.pcd`, courtesy of [this Gist](https://gist.github.com/HTLife/e8f1c4ff1737710e34258ef965b48344).

However, this version has some errors, and thus, I have modified it for your convenience:

```python
def convert_kitti_bin_to_pcd(binFilePath, output_name):
    size_float = 4
    list_pcd = []
    with open(binFilePath, "rb") as f:
        byte = f.read(size_float * 4)
        while byte:
            x, y, z, intensity = struct.unpack("ffff", byte)
            list_pcd.append([x, y, z])
            byte = f.read(size_float * 4)
    np_pcd = np.asarray(list_pcd)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_pcd)
    o3d.io.write_point_cloud(output_name, pcd)

# Data Visualization

In this first part we will observe the 3D pointcloud data. Per the kitty dataset its format is x, y, z, and intensity.  

In [2]:
#Lets first load the data
lidar_data_path = "../DATA/lidar_data/*.pcd"
lidar_file_names = sorted(glob.glob(lidar_data_path))
print(lidar_file_names[0:3])

['../DATA/lidar_data\\0000000000.pcd', '../DATA/lidar_data\\0000000001.pcd', '../DATA/lidar_data\\0000000002.pcd']


In [3]:
#We only need to work with one let us use the first one
lidar_file = lidar_file_names[0]
print(lidar_file)

../DATA/lidar_data\0000000000.pcd


In [4]:
#Read lidar file and create pointcloud data for vizualization
point_cloud = o3d.io.read_point_cloud(lidar_file)

Note: To visualized in open3d there is no need do as below, however this is was done to make the visualizer more asthetically pleasing. To simply visuzlize your point cloud all you need to do is

```python
# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud])

In [5]:
#We will create a function to visuzlize the object with some extra adjustments
def visualize_cloud(obj_list):
    """
    Function modifies the Open3D visualizer for better display. 

    Parameters:
    obj_list (list): A list of objects to visualize this can be multiple point clouds, bounding box data, etc

    Returns:
    type: Void
    """
    
    # Create a visualizer object
    viz = o3d.visualization.Visualizer()
    viz.create_window()
    
    # Add each object to the vizualizer
    for obj in obj_list:
        viz.add_geometry(obj)
    
    #Set the background color to black
    opt = viz.get_render_option()
    opt.background_color = np.array([0, 0, 0])  # RGB values for black
    
    #Run the visualizer
    viz.run()
    viz.destroy_window()
    

Lets call the function the output will be rendered on a different window however its ouput is shown below. 

In [6]:
#Rememeber a list is expected
visualize_cloud([point_cloud])

### Output

![Point Cloud Visualization](../Doc_Images/UNSUPERVISED_DOC_IMAGES/output_1.png)


In the center of the point cloud, a distinct circular pattern is evident. This pattern corresponds to the position of the LiDAR device. A closer examination of the point cloud's structure will provide further understanding of the data characteristics.


In [7]:
#Lets convert the point cloud to a numpy array
point_cloud_numpy = np.asarray(point_cloud.points)
print(point_cloud_numpy.shape)
del point_cloud_numpy

(122595, 3)


### Point Cloud Array Structure

The array is structured as `(N, 3)`, where:

- `N` represents the **total number of points** within the point cloud.
- Each point is defined by a coordinate triplet `(x, y, z)`, which indicates:
  - `x, y, z`: The spatial location of each point in meters.
  - The origin for these coordinates is the LiDAR system lities.
ilities.
itself.

### Additional LiDAR Data Dimensions

In addition to the standard `x, y, z` coordinates, the type of LiDAR system used can provide a range of other data dimensions, including, but not limited to:

- **Intensity**: This measures the reflectivity of surfaces, helping to differentiate materials based on how they reflect light.
- **RGB Color Data**: Offers detailed color information of the scanned surfaces, useful for creating visually rich and accurate representations.
- **Radial Velocity Data**: Available in some advanced LiDAR systems, this feature provides insights into the movement of objects or surfaces, adding a dynamic aspect to the data.

These additional data dimensions significantly enrich the point cloud dataset, allowing for a more comprehensive analysis and enhanced visualization capabilities.


# Plane Segmentation Using Open3D
Open3D offers a comprehensive set of tools for 3D LiDAR point cloud processing, one of which is the plane segmentation feature. This feature employs the RANSAC (Random Sample Consensus) algorithm, a robust method known for its ability to identify and segregate outliers from a dataset. In the context of point clouds, this approach is slightly modified: the 'inliers', or points sharing common characteristics, are typically the ground points, which are often not the focus of our analysis. Consequently, the RANSAC algorithm plays a crucial role in preprocessing by isolating and removing these ground points.

This preprocessing is key to concentrating on the objects of interest, leading to more efficient data processing and enhanced clustering capabilities due to the reduced computational burden.

In our scenario, we have the `point_cloud` variable, an instance of the `PointCloud` class, equipped with the `segment_plane` method. This method is instrumental in excluding ground points and operates based on three parameters, as detailed in the Open3D documentation:

1. `distance_threshold`: Defines the maximum distance a point can be from the estimated plane to be considered an inlier.
2. `ransac_n`: Sets the number of points randomly selected for estimating a plane.
3. `num_iterations`: Specifies the frequency of random plane sampling and evaluation.

The `segment_plane` method outputs the plane's equation in the format `(a, b, c, d)`, conforming to the plane equation `ax + by + cz + d = 0`. This equation is pivotal in differentiating inliers from outliers. Moreover, the method returns a list of indices representing the inlier points, which is invaluable for further data analysis and processing.rocessing.


In [8]:
#Lets start with getting our inlier list and plane equation
plane_model, inliers = point_cloud.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: -0.00x + 0.02y + 1.00z + 1.70 = 0


In [10]:
#Let us visualize the inlier cloud. Remember inliers is a list of indexes of the points that are considered to be inliers by the algorithm
inlier_cloud = point_cloud.select_by_index(inliers)
#Let us paint these points green remember standard RGB is R, G, B
inlier_cloud.paint_uniform_color([0, 1.0, 0])
#Lets now visualize the point cloud
viz.add_geometry(inlier_cloud)
#Run the visualizer
viz.run()
viz.destroy_window()

[Open3D WARNING] GLFW Error: The GLFW library is not initialized
[Open3D WARNING] [PhongShaderForTriangleMesh] Compiling shaders failed.


# DBSCAN clustering

# Voxel downsampling